In [14]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
import utils
from IPython.display import display, Markdown

## directory where results are
# EXPERIMENT_DIR = f"/data/bodyct/experiments/lung-malignancy-fairness-shaurya"
# NLST_PREDS = f"{EXPERIMENT_DIR}/nlst"

TEAMS_DIR = "C:/Users/shaur/OneDrive - Radboudumc/Documents - Master - Shaurya Gaur/General/Malignancy-Estimation Results/nlst"
NLST_PREDS = TEAMS_DIR ## Comment out if not using Teams backup (aka Chansey is up :)

In [4]:
nlst_preds_nodule = pd.read_csv(f"{NLST_PREDS}/nlst_demov4_allmodels_cal.csv")
nlst_preds_nodule.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16077 entries, 0 to 16076
Columns: 123 entries, PatientID to LC_stage
dtypes: bool(25), float64(84), int64(11), object(3)
memory usage: 12.4+ MB


In [12]:
with open(f'{NLST_PREDS}/nlst_demo_v4_cols.json') as json_data:
    nlst_democols = json.load(json_data)
    json_data.close()

nlst_democols['num'].pop('nodule')
nlst_democols

{'num': {'demo': ['height', 'weight', 'Age', 'BMI'],
  'smoke': ['pkyr', 'smokeage', 'smokeday', 'smokeyr'],
  'other': ['NoduleCounts', 'Diameter [mm]'],
  'lungcanc': ['LC_stage']},
 'cat': {'demo': ['marital',
   'educat',
   'race',
   'ethnic',
   'Gender',
   'Married',
   'NonHispanicWhite',
   'HighSchoolPlus'],
  'smoke': ['cigar', 'cigsmok', 'pipe', 'smokelive', 'smokework'],
  'work': ['wrkasbe',
   'wrkbaki',
   'wrkbutc',
   'wrkchem',
   'wrkcoal',
   'wrkcott',
   'wrkfarm',
   'wrkfire',
   'wrkflou',
   'wrkfoun',
   'wrkhard',
   'wrkpain',
   'wrksand',
   'wrkweld'],
  'disease': ['diagadas',
   'diagasbe',
   'diagbron',
   'diagchas',
   'diagchro',
   'diagcopd',
   'diagdiab',
   'diagemph',
   'diagfibr',
   'diaghear',
   'diaghype',
   'diagpneu',
   'diagsarc',
   'diagsili',
   'diagstro',
   'diagtube'],
  'canchist': ['cancblad',
   'cancbrea',
   'canccerv',
   'canccolo',
   'cancesop',
   'canckidn',
   'canclary',
   'canclung',
   'cancnasa',
   'can

In [33]:
cols_num = sum(nlst_democols['num'].values(), start=[])
cols_cat = sum(nlst_democols['cat'].values(), start=[])
cols_list = list(set(cols_num + cols_cat) - set(['LC_stage']))
len(cols_list)

89

In [28]:
nlst_preds = utils.prep_nlst_preds(nlst_preds_nodule, scanlevel=True, tijmen=False, sybil=True)
nlst_preds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5911 entries, 0 to 10150
Columns: 118 entries, PatientID to LC_stage
dtypes: bool(25), float64(81), int64(11), object(1)
memory usage: 4.4+ MB


In [29]:
MODEL_TO_COL = {
    "Venkadesh": "DL_cal",
    # "de Haas Combined": "Thijmen_mean",
    "de Haas Local": "Thijmen_local_cal",
    "de Haas Global (hidden nodule)": "Thijmen_global_hidden_cal",
    "de Haas Global (shown nodule)": "Thijmen_global_show_cal",
    "PanCan2b": "PanCan2b",
    "Sybil year 1": "sybil_year1",
    # "Sybil year 2": "sybil_year2",
    # "Sybil year 3": "sybil_year3",
    # "Sybil year 4": "sybil_year4",
    # "Sybil year 5": "sybil_year5",
    # "Sybil year 6": "sybil_year6",
}

## What is a factor for each model?

In [47]:
nlst_preds2 = nlst_preds.dropna(axis=0, subset=cols_list)
nlst_preds2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5570 entries, 0 to 10150
Columns: 118 entries, PatientID to LC_stage
dtypes: bool(25), float64(81), int64(11), object(1)
memory usage: 4.1+ MB


In [48]:
X = nlst_preds2[['label'] + cols_list]
sm.add_constant(X)

,const,label,wrkhard,canccolo,cancpanc,Age,Emphysema,diagdiab,cancnasa,diagcopd,...,wrknomask,FamilyHistoryLungCa,wrkfoun,smokeage,cancblad,Gender,cancbrea,Adenocarcinoma,diagtube,canckidn
0,1.0,0,0.0,0.0,0.0,62,True,0.0,0.0,0.0,...,False,False,0.0,15.0,0.0,1,0.0,False,0.0,0.0
1,1.0,0,0.0,0.0,0.0,56,True,0.0,0.0,0.0,...,False,False,0.0,17.0,0.0,2,0.0,False,0.0,0.0
2,1.0,0,0.0,0.0,0.0,57,True,0.0,0.0,0.0,...,False,False,0.0,17.0,0.0,2,0.0,False,0.0,0.0
3,1.0,0,0.0,0.0,0.0,61,False,0.0,0.0,0.0,...,True,False,0.0,15.0,0.0,1,0.0,False,0.0,0.0
4,1.0,0,0.0,0.0,0.0,62,False,0.0,0.0,0.0,...,True,False,0.0,15.0,0.0,1,0.0,False,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10119,1.0,1,0.0,0.0,0.0,66,True,1.0,0.0,0.0,...,True,False,1.0,17.0,0.0,1,0.0,False,0.0,0.0
10120,1.0,1,0.0,0.0,0.0,67,True,1.0,0.0,0.0,...,True,False,1.0,17.0,0.0,1,0.0,False,0.0,0.0
10121,1.0,1,0.0,0.0,0.0,68,False,1.0,0.0,0.0,...,True,False,1.0,17.0,0.0,1,0.0,False,0.0,0.0
10130,1.0,0,0.0,0.0,0.0,58,True,0.0,0.0,1.0,...,True,False,0.0,17.0,0.0,1,0.0,False,0.0,0.0


In [51]:
for m in MODEL_TO_COL:
    y = nlst_preds2[MODEL_TO_COL[m]]
    display(Markdown(f"### {m}"))
    est = sm.OLS(y, X.astype(float))
    est2 = est.fit()
    print(est2.summary())

### Venkadesh

                                 OLS Regression Results                                
Dep. Variable:                 DL_cal   R-squared (uncentered):                   0.658
Model:                            OLS   Adj. R-squared (uncentered):              0.652
Method:                 Least Squares   F-statistic:                              118.3
Date:                Wed, 29 Jan 2025   Prob (F-statistic):                        0.00
Time:                        21:14:01   Log-Likelihood:                          4374.3
No. Observations:                5570   AIC:                                     -8571.
Df Residuals:                    5481   BIC:                                     -7981.
Df Model:                          89                                                  
Covariance Type:            nonrobust                                                  
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------

### de Haas Local

                                 OLS Regression Results                                
Dep. Variable:      Thijmen_local_cal   R-squared (uncentered):                   0.627
Model:                            OLS   Adj. R-squared (uncentered):              0.621
Method:                 Least Squares   F-statistic:                              103.5
Date:                Wed, 29 Jan 2025   Prob (F-statistic):                        0.00
Time:                        21:14:01   Log-Likelihood:                          4101.1
No. Observations:                5570   AIC:                                     -8024.
Df Residuals:                    5481   BIC:                                     -7435.
Df Model:                          89                                                  
Covariance Type:            nonrobust                                                  
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------

### de Haas Global (hidden nodule)

...

                                   OLS Regression Results                                   
Dep. Variable:     Thijmen_global_show_cal   R-squared (uncentered):                   0.626
Model:                                 OLS   Adj. R-squared (uncentered):              0.620
Method:                      Least Squares   F-statistic:                              103.0
Date:                     Wed, 29 Jan 2025   Prob (F-statistic):                        0.00
Time:                             21:14:01   Log-Likelihood:                          4378.7
No. Observations:                     5570   AIC:                                     -8579.
Df Residuals:                         5481   BIC:                                     -7990.
Df Model:                               89                                                  
Covariance Type:                 nonrobust                                                  
                                    coef    std err          t      P>

### PanCan2b

                                 OLS Regression Results                                
Dep. Variable:               PanCan2b   R-squared (uncentered):                   0.905
Model:                            OLS   Adj. R-squared (uncentered):              0.903
Method:                 Least Squares   F-statistic:                              585.3
Date:                Wed, 29 Jan 2025   Prob (F-statistic):                        0.00
Time:                        21:14:01   Log-Likelihood:                          10285.
No. Observations:                5570   AIC:                                 -2.039e+04
Df Residuals:                    5481   BIC:                                 -1.980e+04
Df Model:                          89                                                  
Covariance Type:            nonrobust                                                  
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------

### Sybil year 1

                                 OLS Regression Results                                
Dep. Variable:            sybil_year1   R-squared (uncentered):                   0.483
Model:                            OLS   Adj. R-squared (uncentered):              0.475
Method:                 Least Squares   F-statistic:                              57.59
Date:                Wed, 29 Jan 2025   Prob (F-statistic):                        0.00
Time:                        21:14:02   Log-Likelihood:                          6631.3
No. Observations:                5570   AIC:                                 -1.308e+04
Df Residuals:                    5481   BIC:                                 -1.250e+04
Df Model:                          89                                                  
Covariance Type:            nonrobust                                                  
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------

## What about by demographic characteristics?

In [ ]:
confound_cols = list(set(cols_list) - set(nlst_democols['cat']['demo']))
nlst_preds3 = nlst_preds.dropna(axis=0, subset=confound_cols)
# X_confound = nlst_preds3[['label'] + confound_cols]
# sm.add_constant(X_confound)

,const,label,wrkhard,canccolo,cancpanc,Age,Emphysema,diagdiab,cancnasa,diagcopd,...,diagsarc,wrknomask,FamilyHistoryLungCa,wrkfoun,smokeage,cancblad,cancbrea,Adenocarcinoma,diagtube,canckidn
0,1.0,0,0.0,0.0,0.0,62,True,0.0,0.0,0.0,...,0.0,False,False,0.0,15.0,0.0,0.0,False,0.0,0.0
1,1.0,0,0.0,0.0,0.0,56,True,0.0,0.0,0.0,...,0.0,False,False,0.0,17.0,0.0,0.0,False,0.0,0.0
2,1.0,0,0.0,0.0,0.0,57,True,0.0,0.0,0.0,...,0.0,False,False,0.0,17.0,0.0,0.0,False,0.0,0.0
3,1.0,0,0.0,0.0,0.0,61,False,0.0,0.0,0.0,...,0.0,True,False,0.0,15.0,0.0,0.0,False,0.0,0.0
4,1.0,0,0.0,0.0,0.0,62,False,0.0,0.0,0.0,...,0.0,True,False,0.0,15.0,0.0,0.0,False,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10119,1.0,1,0.0,0.0,0.0,66,True,1.0,0.0,0.0,...,0.0,True,False,1.0,17.0,0.0,0.0,False,0.0,0.0
10120,1.0,1,0.0,0.0,0.0,67,True,1.0,0.0,0.0,...,0.0,True,False,1.0,17.0,0.0,0.0,False,0.0,0.0
10121,1.0,1,0.0,0.0,0.0,68,False,1.0,0.0,0.0,...,0.0,True,False,1.0,17.0,0.0,0.0,False,0.0,0.0
10130,1.0,0,0.0,0.0,0.0,58,True,0.0,0.0,1.0,...,0.0,True,False,0.0,17.0,0.0,0.0,False,0.0,0.0


In [55]:
nlst_preds3['bmi_classes'] = pd.cut(nlst_preds3['BMI'], bins=[0, 
                                                            # 18.5, 
                                                            25, 
                                                            # 30, 
                                                            100], right=False)

C:\Users\shaur\AppData\Local\Temp\ipykernel_14908\1916830064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nlst_preds3['bmi_classes'] = pd.cut(nlst_preds3['BMI'], bins=[0,


In [56]:
for c in ['Gender', 'race', 'bmi_classes']:
    groups = nlst_preds3.groupby(c)
    
    for val, dfg in groups:
        display(Markdown(f"### {c} = {val}"))
        Xg = dfg[['label'] + confound_cols]
        sm.add_constant(Xg)

        for m in MODEL_TO_COL:
            y = dfg[MODEL_TO_COL[m]]
            display(Markdown(f"#### {m}"))
            est = sm.OLS(y, Xg.astype(float))
            est2 = est.fit()
            print(est2.summary())

### Gender = 1

#### Venkadesh

                                 OLS Regression Results                                
Dep. Variable:                 DL_cal   R-squared (uncentered):                   0.659
Model:                            OLS   Adj. R-squared (uncentered):              0.651
Method:                 Least Squares   F-statistic:                              82.48
Date:                Wed, 29 Jan 2025   Prob (F-statistic):                        0.00
Time:                        21:25:33   Log-Likelihood:                          2667.4
No. Observations:                3360   AIC:                                     -5181.
Df Residuals:                    3283   BIC:                                     -4710.
Df Model:                          77                                                  
Covariance Type:            nonrobust                                                  
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------

#### de Haas Local

                                 OLS Regression Results                                
Dep. Variable:      Thijmen_local_cal   R-squared (uncentered):                   0.628
Model:                            OLS   Adj. R-squared (uncentered):              0.619
Method:                 Least Squares   F-statistic:                              71.88
Date:                Wed, 29 Jan 2025   Prob (F-statistic):                        0.00
Time:                        21:25:33   Log-Likelihood:                          2389.2
No. Observations:                3360   AIC:                                     -4624.
Df Residuals:                    3283   BIC:                                     -4153.
Df Model:                          77                                                  
Covariance Type:            nonrobust                                                  
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------

...

                                   OLS Regression Results                                   
Dep. Variable:     Thijmen_global_show_cal   R-squared (uncentered):                   0.617
Model:                                 OLS   Adj. R-squared (uncentered):              0.609
Method:                      Least Squares   F-statistic:                              77.97
Date:                     Wed, 29 Jan 2025   Prob (F-statistic):                        0.00
Time:                             21:25:37   Log-Likelihood:                          3165.2
No. Observations:                     3910   AIC:                                     -6172.
Df Residuals:                         3831   BIC:                                     -5677.
Df Model:                               79                                                  
Covariance Type:                 nonrobust                                                  
                                    coef    std err          t      P>

#### PanCan2b

                                 OLS Regression Results                                
Dep. Variable:               PanCan2b   R-squared (uncentered):                   0.903
Model:                            OLS   Adj. R-squared (uncentered):              0.901
Method:                 Least Squares   F-statistic:                              453.2
Date:                Wed, 29 Jan 2025   Prob (F-statistic):                        0.00
Time:                        21:25:37   Log-Likelihood:                          7538.1
No. Observations:                3910   AIC:                                 -1.492e+04
Df Residuals:                    3831   BIC:                                 -1.442e+04
Df Model:                          79                                                  
Covariance Type:            nonrobust                                                  
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------

#### Sybil year 1

                                 OLS Regression Results                                
Dep. Variable:            sybil_year1   R-squared (uncentered):                   0.482
Model:                            OLS   Adj. R-squared (uncentered):              0.472
Method:                 Least Squares   F-statistic:                              45.17
Date:                Wed, 29 Jan 2025   Prob (F-statistic):                        0.00
Time:                        21:25:37   Log-Likelihood:                          4894.4
No. Observations:                3910   AIC:                                     -9631.
Df Residuals:                    3831   BIC:                                     -9135.
Df Model:                          79                                                  
Covariance Type:            nonrobust                                                  
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------